In [1]:
from text_preprocessor import CorpusToEmbedding

import pandas as pd

df = pd.read_csv("./preprocessed/train.csv")

import pickle

with open("preprocessed/text_object", 'rb') as f:
    text_obj = pickle.load(f)
    f.close()

In [2]:
df["text_tokens"] = text_obj.tokens
df["labels_tokens"] = text_obj.labels

In [3]:
df

,Unnamed: 0,id,img,label,text,labels,text_tokens,labels_tokens
0,0,42953,img/42953.png,0,its their character not their color that matters,"['person', 'person']","[19, 45, 730, 34, 45, 1549, 11, 619, 999999, 9...","[361, 361, 999999]"
1,1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...,"['person', 'person', 'person', 'tie']","[98, 34, 38, 4840, 5, 650, 177, 584, 13, 34, 6...","[361, 361, 361]"
2,2,13894,img/13894.png,0,putting bows on your pet,"['cat', 'cat', 'bed']","[1629, 8139, 15, 54, 8544, 999999, 999999, 999...","[3511, 3511, 2818]"
3,3,37408,img/37408.png,0,i love everything and everybody! except for sq...,"['dog', 'dog']","[650, 749, 3, 3409, 1231, 14, 23171, 2992, 231...","[2369, 2369, 999999]"
4,4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h...","['person', 'tie', 'tie']","[3409, 650, 7037, 7027, 435739, 134, 60393, 99...","[361, 4540, 4540]"
...,...,...,...,...,...,...,...,...
8495,8495,10423,img/10423.png,1,nobody wants to hang auschwitz me,"['person', 'tie', 'tie', 'bed', 'bed', 'bed']","[4014, 405, 5, 8974, 448544, 103, 999999, 9999...","[361, 4540, 4540]"
8496,8496,98203,img/98203.png,1,when god grants you a child after 20 years of ...,"['person', 'person', 'person', 'person', 'pers...","[71, 3730, 2590, 30, 815, 79, 163, 144, 4, 54,...","[361, 361, 361]"
8497,8497,36947,img/36947.png,1,gays on social media: equality! body positivit...,['person'],"[2635, 15, 687, 3394, 4683, 561, 41234, 650, 6...","[361, 999999, 999999]"
8498,8498,16492,img/16492.png,1,having a bad day? you could be a siamese twin ...,['person'],"[282, 816, 193, 30, 169, 38, 282629, 8287, 462...","[361, 999999, 999999]"


In [4]:
from torchvision import datasets, transforms
import torch

/home/vincent/miniforge3/envs/hateful-memes-classification/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/vincent/miniforge3/envs/hateful-memes-classification/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [5]:
import numpy as np

df["new_labels"] = np.where(df.label == 0, 1, 0)
df["new_id"] = df.id.astype(str) # for lexsorting

In [6]:
df = df.sort_values(["new_labels", "new_id"])

In [7]:
texts = torch.utils.data.TensorDataset(torch.tensor(df.text_tokens), torch.tensor(df.new_labels))

In [8]:
labels = torch.utils.data.TensorDataset(torch.tensor(df.labels_tokens), torch.tensor(df.new_labels))

In [9]:
transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

images = datasets.ImageFolder("./preprocessed/img", transform=transformations)

In [10]:
from torch.utils.data.sampler import RandomSampler

random_sampler = RandomSampler(images)

In [11]:
image_loader = torch.utils.data.DataLoader(
    images, batch_size=8, sampler=random_sampler
)
text_loader = torch.utils.data.DataLoader(texts, batch_size=8, sampler=random_sampler)

labels_loader = torch.utils.data.DataLoader(labels, batch_size=8, sampler=random_sampler)

In [12]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        drp = 0.1
        n_classes = 2

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)

        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(text_obj.weights))
        self.embedding.requires_grad = False

        self.lstm = nn.LSTM(300, 30, bidirectional=False, batch_first=True)
        self.text_linear = nn.Sequential(
            nn.Linear(30, 15),
            nn.Linear(15, n_classes)
        )

        self.cnn_layer1 = nn.Sequential(
            # Layer 1
            nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(drp),
            # Layer 2
            nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(drp),
            # # Layer 3
            # nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=0),
            # # Out: 62, 29
            # nn.BatchNorm2d(32),
            # nn.ReLU(inplace=True),
            # nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
            # # Out: 30, 14
            # nn.Dropout(drp),
        )

        self.fc1 = nn.Linear(224 * 224 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)
        self.out = nn.Linear(64, n_classes)

    def forward(self, x):
        # x = torch.unsqueeze(x, dim=1)
        x = x[1]
        h_embedding = self.embedding(x)
        h_lstm, _ = self.lstm(h_embedding)
        out = self.text_linear(h_lstm)
        # out1 = self.cnn_layer1(x)
        # out2 = self.cnn_layer2(x)
        # full_out = torch.cat([out1, out2], dim=1)
        # full_out = full_out.view(full_out.size(0), -1)
        # x = torch.flatten(out1, start_dim=1)
        # x = self.dropout(self.relu(self.fc1(x)))
        # x = self.dropout(self.relu(self.fc2(x)))
        # x = self.dropout(self.relu(self.fc3(x)))
        # x = self.dropout(self.relu(self.fc4(x)))
        # x = self.dropout(self.relu(self.fc5(x)))
        # out = self.out(x)
        return out

In [21]:
class Model(nn.Module):
    def __init__(self, weights) -> None:
        super().__init__()

        self.drp = 0.1
        self.n_classes = 2

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(self.drp)
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(weights))

        # Text LSTM
        self.lstm = nn.LSTM(
            300, 30, bidirectional=False, batch_first=True, num_layers=3, dropout=self.drp
        )

        # LSTM linear
        self.lstm_linear = nn.Sequential(
            nn.Linear(900, 450),
            nn.ReLU(inplace=True),
            nn.Linear(450, 200),
            nn.ReLU(inplace=True),
        )

        # Text CNN
        self.text_cnn = nn.Sequential(
            # Layer 1
            nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(self.drp),
            # Layer 2
            nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(self.drp),
            # Layer 3
            nn.Conv2d(3, 3, kernel_size=3, stride=(1, 10), padding=(1, 0)),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(self.drp),
        )

        # Image CNN
        self.image_cnn = nn.Sequential(
            # Layer 1
            nn.Conv2d(3, 3, kernel_size=3, stride=3, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(self.drp),
            # Layer 2
            nn.Conv2d(3, 3, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=0),
            nn.Dropout(self.drp),
            # Layer 3
            nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=5, stride=1, padding=0),
            nn.Dropout(self.drp),
        )

        # Image + Text CNN
        self.combined_cnn = nn.Sequential(
            # Layer 1
            nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(self.drp),
        )

        # Labels linear
        self.labels_linear = nn.Sequential(
            nn.Linear(900, 450),
            nn.ReLU(inplace=True),
            nn.Linear(450, 200),
            nn.ReLU(inplace=True),
        )

        # All linear

        self.all_linear = nn.Sequential(
            nn.Linear(3100, 1200),
            nn.ReLU(inplace=True),
            nn.Linear(1200, 600),
            nn.ReLU(inplace=True),
            nn.Linear(600, 300),
            nn.ReLU(inplace=True),
            nn.Linear(300, 150),
            nn.ReLU(inplace=True),
            nn.Linear(150, 75),
            nn.ReLU(inplace=True),
            nn.Linear(75, 2),
        )
    
    def forward(self, x):
        img, text, labels = x

        # Process image
        cnn_img = self.image_cnn(img)

        # Process text
        text_emb = self.embedding(text)
        lstm, _ = self.lstm(text_emb)
        lstm_single = lstm.reshape(lstm.shape[0], -1)
        lstm_out = self.lstm_linear(lstm_single)
        text_emb_cnn = torch.unsqueeze(text_emb, 1)
        cnn_text = self.text_cnn(text_emb_cnn)

        # Add text and image CNN
        combined_cnn = cnn_img + cnn_text
        combined_cnn = self.combined_cnn(combined_cnn)
        combined_linear = combined_cnn.view(combined_cnn.shape[0], -1)

        # Process labels
        labels_emb = self.embedding(labels)
        labels_single = labels_emb.view(labels.shape[0], 900)
        labels_out = self.labels_linear(labels_single)

        # Process all
        combined_all = torch.cat((lstm_out, combined_linear, labels_out), dim=1)
        out = self.all_linear(combined_all)

        return out

In [22]:
model = Model(text_obj.weights)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)

# loss_fn = nn.

for (i, j, k) in zip(image_loader, text_loader, labels_loader):
    image = i[0].to(device)
    text = j[0].to(device)
    labels = k[0].to(device)
    output = model((image, text, labels)) # hacky way to do this?
    print(output)
    break

tensor([[0.0300, 0.0356],
        [0.0288, 0.0259],
        [0.0318, 0.0325],
        [0.0234, 0.0269],
        [0.0274, 0.0250],
        [0.0268, 0.0285],
        [0.0267, 0.0281],
        [0.0142, 0.0274]], device='cuda:0', grad_fn=<AddmmBackward0>)
